# Введение в генеративные модели

В данном ноутбуке будет рассмотрено несколько базовых подходов к генерации изображений с помощью сверточных нейросетей. Мы будем генерировать фотографии лиц. В качестве эталона, будем использовать выровненные, центрированные фотографии знаменитостей из датасета CelebFaces Attributes (CelebA).

Установим нужные пакеты, скачаем датасет и чекпоинты моделей, которые нам понадобятся в этом задании.

In [ ]:
!pip install --quiet matplotlib torchvision lightning-bolts gdown

In [ ]:
import glob
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pl_bolts as bolt
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchvision as tv
from IPython.display import display
from PIL import Image
from pl_bolts.models.autoencoders.components import resnet18_decoder, resnet18_encoder

In [ ]:
try:
    # Download, verify and extract the CelebA dataset
    tv.datasets.CelebA(".", download=True)
except Exception:
    print("Official download method failed. Trying direct download from Google Drive.")
    !mkdir -p celeba
    !gdown --id 1bddRLZ8KpwR_gOYRS_FcQ7FLrkFCs39L --output celeba/img_align_celeba.zip
    !gdown --id 1k3O3jGswTH09UytdkoXSxblj6RzpdzQN --output celeba_txt.zip
    !unzip -o celeba_txt.zip

    print()
    for _, md5sum, file_name in tv.datasets.CelebA.file_list:
        print("Expected:  ", md5sum, " celeba/" + file_name)
        print("Downloaded: ", end="", flush=True)
        !md5sum "celeba/{file_name}"
        print()

    # Verify and extract the CelebA dataset
    tv.datasets.CelebA(".", download=True)

# Download pretrained model checkpoints
if not os.path.exists("AutoEncoder.ckpt"):
    !gdown --id 1nCNYSY0hHq6ZejLEIGO4Xy4k8wqZC084 --output AutoEncoder.ckpt
if not os.path.exists("VariationalAutoEncoder.ckpt"):
    !gdown --id 1f04WZXkVokaotdDyyiTXJHtRvTl-qMdJ --output VariationalAutoEncoder.ckpt
if not os.path.exists("GenerativeAdversarialNetwork.ckpt"):
    !gdown --id 1KZ_FtfHrwM0LCul3pdArhjnDzbwd7B_8 --output GenerativeAdversarialNetwork.ckpt

In [ ]:
# This cell is mostly boilerplate code for working with the CelebA dataset.
class CelebABoilerplate(pl.LightningModule):
    def __init__(self, image_size, batch_size=64, lr=0.0001, **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()

    def get_dataset(self, kind):
        return tv.datasets.CelebA(
            ".",
            split=kind,
            transform=tv.transforms.Compose([
                tv.transforms.Resize(self.hparams.image_size),
                tv.transforms.CenterCrop(self.hparams.image_size),
                tv.transforms.ToTensor(),
            ]),
        )

    def get_dataloader(self, kind):
        return torch.utils.data.DataLoader(
            self.get_dataset(kind),
            num_workers=os.cpu_count(),
            shuffle=kind == "train",
            batch_size=self.hparams.batch_size,
        )
    
    def train_dataloader(self):
        return self.get_dataloader("train")

    def val_dataloader(self):
        return self.get_dataloader("valid")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

def get_latest_version(root_dir):
    versions = sorted(glob.glob(f"{root_dir}/lightning_logs/version_*"))
    return versions[-1] if versions else None

def get_latest_checkpoint(root_dir):
    last_version = get_latest_version(root_dir)
    checkpoints = sorted(glob.glob(f"{last_version}/checkpoints/*")) if last_version else []
    return checkpoints[-1] if checkpoints else None

def show_images(imgs, scale=2):
    imgs = np.concatenate([
        np.concatenate([
            (255 * img.transpose((1, 2, 0))).round().clip(0, 255).astype(np.uint8)
            for img in row
        ], axis=1)
        for row in imgs
    ], axis=0)

    display(Image.fromarray(imgs, "RGB").resize([scale * d for d in imgs.shape[:2][::-1]]))

warnings.filterwarnings("ignore", message="Named tensors and all their associated APIs are an experimental feature and subject to change.")

---

## Часть 1: Автоэнкодеры

Автоэнкодеры (autoencoder, AE, автокодировщик) - особый вид нейросетевых архитектур, состоящих из двух последовательных подсетей, называемых энкодером и декодером. Задача энкодера - понизить размерность входных данных, а задача декодера - восстановить исходные данные по выходу энкодера.

Обучение автоэкодера заключается в минимизации "ошибки рекострукции" - расстояния между исходными входными данными и восстановленными декодером данными. Достаточно часто в качестве "расстояния" берется просто поэлементное среднеквадратичное отклонение, однако в общем случае можно минимизировать и любое другое расстояние.

Если декодер успешно восстанавливает входные данные, то можно считать, что промежуточные низкоразмерные данные являются особым *представлением* исходных данных. Такое "сжатое" представление называется "латентным", а исходное представление данных - "естественным".

Важно понимать, что понижение размерности возможно только за счет *избыточностей* в естественном представлении целевых данных. Например, цветные фотографии лиц знаменитостей можно естественно представить как элементы в $\mathbb{R}^{H{\times}W{\times}3}$, однако данное представление очевидно избыточно. Подавляющее большинство элементов $\mathbb{R}^{H{\times}W{\times}3}$ выглядит как случайный шум или как какие-то другие изображения (не лиц знаменитостей).

С другой стороны, интуиция подсказывает нам, что пространство фотографий лиц знаменитостей не дискретно, а обладает некоторой "локальностью" (*небольшие* изменения пикселей фотографии не приводят к резкому изменению свойств фотографии). Отсюда следует логичное предположение, что латентное представление тоже будет обладать этим свойством.

![ae.png](https://docs.google.com/uc?export=download&id=1sCME120ySUHrW3_Gsi-9fHtJyoOaVsQ9)

### Детали реализации

Обычно в случае работы с изображениями, в качестве сети-энкодера используется какая-нибудь "стандартная" сверточная нейросеть, сжимающая входное изображения в "плотный" вектор признаков. Архитектура сети-декодера при этом зачастую составляет "отраженную" архитектуру энкодера, в которой все слои идут в обратном порядке и вместо постепенного уменьшение разрешения, происходит его увеличение.

Для повышения разрешения изображения можно заменять слои по следующему принципу. <br/>
Вместо [усреднения/пулинга (`AvgPool2d`)](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html) использовать [интерполяцию/масштабирование (`Upsample`)](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html). <br/>
Вместо [сверток (`Conv2d`)](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) с шагом (`stride > 1`) использовать [транспонированные свертки (`ConvTranspose2d`)](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html) с тем же шагом.

Здесь и в последующих заданиях в качестве архитектур энкодера/декодера используется модифицированная версия ResNet-18.

Внимательно рассмотрите код предоставленной реализации простого сверточного автоэнкодера. <br/>
Особое внимание обратите на функции `forward` и `step`.

In [ ]:
class CelebAAutoEncoder(CelebABoilerplate):
    def __init__(self, first_conv=False, maxpool1=False, latent_dim=512, **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()
        self.encoder = resnet18_encoder(self.hparams.first_conv, self.hparams.maxpool1)
        self.decoder = resnet18_decoder(self.hparams.latent_dim, self.hparams.image_size, self.hparams.first_conv, self.hparams.maxpool1)

    def forward(self, x):
        z = self.encoder(x)
        x_rec = self.decoder(z)
        return z, x_rec

    def step(self, batch, batch_idx):
        kind = "train" if self.training else "valid"
        x, _ = batch
        z, x_rec = self.forward(x)
        
        # Reconstruction loss
        loss = F.mse_loss(x, x_rec)
        self.log(f"{kind}_loss", loss, prog_bar=True)
        
        return loss

    def training_step(self, *args, **kwargs):
        return self.step(*args, **kwargs)
    
    def validation_step(self, *args, **kwargs):
        return self.step(*args, **kwargs)

Для экономии времени на семинаре вам предлагается загрузить заранее обученную версию модели. <br/>
Также для справки приведен код, который был использован для обучения этой модели.

In [ ]:
# Create an AutoEncoder
if False:
    ae = CelebAAutoEncoder(image_size=56)
    trainer = pl.Trainer(
        auto_lr_find=True,
        gpus=1,
        log_gpu_memory=True,
        callbacks=[pl.callbacks.EarlyStopping("valid_loss", patience=2, verbose=True)],
        default_root_dir="AutoEncoder",
        resume_from_checkpoint=get_latest_checkpoint("AutoEncoder"),
    )
    # Tune the learning rate
    trainer.tune(ae)['lr_find'].plot()
    plt.show()
    # Train the model
    trainer.fit(ae)
else:
    ae = CelebAAutoEncoder.load_from_checkpoint("AutoEncoder.ckpt")

### Визуализация результатов

Давайте рассмотрим результаты работы обученного автоэнкодера. Для этого визуализируем исходные и восстановленные изображения.

In [ ]:
def show_autoencoder_results(ae, num_images=10):
    ae.train(False)
    valid_ds = ae.get_dataset("valid")

    inps = []
    outs = []
    for i in np.random.choice(len(valid_ds), size=num_images):
        inp, _ = valid_ds[i]
        _, out, *_ = ae.forward(inp[np.newaxis])
        out = out.squeeze(0).detach()
        inps.append(inp.numpy())
        outs.append(out.numpy())

    show_images([inps, outs])

In [ ]:
show_autoencoder_results(ae)

**❓Ответьте на вопросы:** Похожи ли восстановленные изображения на исходные?

### Свойства латентного пространства

Также, давайте проверим нашу гипотезу о локальности латентного пространства. Для этого вычислим латентные представления для двух случайных изображений и линейно проинтерполируем несколько промежуточных латентных представлений. Декодируем и визуализируем эти латентные представления.

In [ ]:
def show_autoencoder_interpolation(ae, num_images=10):
    ae.train(False)
    valid_ds = ae.get_dataset("valid")

    i0, i1 = np.random.choice(len(valid_ds), size=2)
    img0, _ = valid_ds[i0]
    img1, _ = valid_ds[i1]
    z0, *_ = ae.forward(img0[np.newaxis])
    z1, *_ = ae.forward(img1[np.newaxis])
    
    inps = [img0.numpy()] + (num_images-2) * [np.zeros(img0.shape)] + [img1.numpy()]
    outs = []
    for lerp in np.linspace(0, 1, num_images):
        out = ae.decoder((1-lerp) * z0 + lerp * z1).squeeze(0).detach()
        outs.append(out.numpy())

    show_images([inps, outs])

In [ ]:
show_autoencoder_interpolation(ae)

**❓Ответьте на вопросы:** Оправдалось ли наше предположение о локальности латентного пространства?

### Генерация новых изображений с помощью автоэнкодера

Возвращаясь к теме семинара, давайте подумаем, можно ли как-нибудь использовать получившееся латентное представление и его свойства для генерации новых изображений.

Как было упомянуто ранее, нам известно, что множество "фотографии лиц знаменитостей" как подпространство $\mathbb{R}^{H{\times}W{\times}3}$ составляет крайне малую долю из всех возможных изображений. Из-за этого, мы очевидно не можем просто сгенерировать матрицу случайных значений размера $H{\times}W{\times}3$ и ожидать, что это будет фотографией лица.

С другой стороны, наше латентное пространство имеет сильно меньшую размерность, а значит разумно предположить, что множество "фотографии лиц знаменитостей" в нем будет более "плотно" расположено. Давайте попробуем визуализировать результаты декодирования для случайных латентных векторов.

Для простоты, предположим также что латентные вектора состоят из нормально распределенных случайных величин. Вычислим среднее значение и среднеквадратичное отклонение элементов латентных векторов, сгенерируем с помощью этих параметров новые случайные латентные вектора и визуализируем их.

In [ ]:
def get_mean_std(ae, num_samples=1000):
    ae.train(False)
    valid_ds = ae.get_dataset("valid")

    zs = []
    for i in np.random.choice(len(valid_ds), size=num_samples):
        inp, _ = valid_ds[i]
        z, *_ = ae.forward(inp[np.newaxis])
        out = z.squeeze(0).detach()
        zs.append(out.numpy())

    mean = np.mean(zs, axis=0)
    std = np.std(zs, axis=0)
    return mean, std


def show_decode_random(ae, mean, std, num_images=10):
    ae.train(False)

    mean = torch.from_numpy(mean).to(ae.device)
    std = torch.from_numpy(std).to(ae.device)
    imgs = []
    for i in range(num_images):
        imgs.append([])
        for k in range(num_images):
            z = mean + std * torch.randn(ae.hparams.latent_dim).to(ae.device)
            out = ae.decoder(z[np.newaxis]).squeeze(0).detach()
            imgs[-1].append(out.numpy())

    show_images(imgs)

In [ ]:
mean, std = get_mean_std(ae)
show_decode_random(ae, mean, std)

**❓Ответьте на вопросы:** Хорошие ли получились изображения? Если нет, попробуйте объяснить в чем проблема. Какое из наших предположений не оправдалось (о "локальности" латентного представления, о "плотности" латентного представления или о нормальности распределения латентных векторов)?

In [ ]:
# Free up some memory for the next section
del ae

---

## Часть 2: Вариационные автоэнкодеры

Вариационные автоэнкодеры (variational autoencoders, VAE) - модификация стандартных автоэнкодеров, нацеленная на улучшение свойств латентного представления с целью генерации правдоподобных новых данных.

Как можно было догадаться из последнего задания, предложенный метод генерации новых данных с помощью стандартных автоэнкодеров работал плохо из-за того что не выполнялось одно из сделанных нами предположений. А именно, латентные вектора в стандартных автоэнкодерах **не** распределены нормально.

Действительно, без каких либо ограничений на латентное представление нет оснований полагать, что элементы вектора будут иметь нужное нам распределение. Теперь, когда мы знаем, в чем была проблема стандартного автоэнкодера, рассмотрим как эту проблему предлагается решить в вариационных автоэнкодерах.

Вместо того чтобы предсказывать латентный вектор напрямую, вариационный энкодер предсказывает параметры для распределения и затем из этого распределения сэмплируется случайный латентный вектор. Полученный случайный латентный вектор, как и в стандартном автоэнкодере, должен декодироваться в исходное изображение. Таким образом, латентные вектора гарантированно будут иметь нужное нам распределение (по построению).

![vae.png](https://docs.google.com/uc?export=download&id=1N1yUxjB32gN2EYI3efKSseHyY3xjJNnI)

Концептуально, идея вариационных автоэнкодеров получается достаточно простая, однако на практике есть ещё 3 важных нюанса, которые необходимо учесть для получения работающего решения.

### $N$-мерность пространства латентных векторов

Особо внимательные студенты могли заметить, что говоря о распределениях латентных представлений мы намеренно опускали одну важную деталь - латентные представления являются $N$-мерными векторами, а не скалярами.

В частности, это значит что элементы этих векторов могут быть не независимо распределены, а "нормальное распределение" должно принять вид $\mathcal{N}\left(\mu,\,\Sigma\right)$, где $\mu \in \mathbb{R}^N$ - вектор сдвига, а $\Sigma \in \mathbb{R}^{N{\times}N}$ - ковариационная матрица. Использование произвольной ковариационной матрицы значительно усложняет дальнейшие выводы, поэтому обычно вводится дополнительное ограничение $\Sigma = \mathrm{I}\sigma$, где $\sigma \in \mathbb{R}^N_+$ - вектор среднеквадратичных отклонений.

Если исходная матрица $\Sigma$ была диагонализируема, то данное ограничение даже не приводит к потере общности (не диагональный множитель можно "занести" в первый линейный слой в декодере).

### Регуляризация распределения

В вариационном автоэнкодере мы заменили предсказание конкретных латентных значений на моделирование их распределения. Однако в текущей версии нашего алгоритма, энкодер может "обойти" наше требование просто предсказывая очень большие значения $\mu$, либо очень маленькие значения $\sigma$. В результате предсказанные распределения будут вести себя почти как обыкновенные "точечные" предсказания в стандартном автоэнкодере.

Чтобы вырождения распределений не происходило, необходимо регуляризовать предсказанные энкодером параметры распределений.

![regularization.png](https://docs.google.com/uc?export=download&id=1jR-TzxqG5v1cgsTOpj6XTgPAPMpRCr4S)

Для регуляризации параметров распределений в вариационном автоэнкодере, предлагается добавить в функцию потерь специальное слагаемое, измеряющее степень близости предсказанного распределения к стандартному.

Как вам известно, [дивергенция Кульбака — Лейблера](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9A%D1%83%D0%BB%D1%8C%D0%B1%D0%B0%D0%BA%D0%B0_%E2%80%94_%D0%9B%D0%B5%D0%B9%D0%B1%D0%BB%D0%B5%D1%80%D0%B0) задает аналог "расстояния" или меры удаленности одного распределения от другого.

В данном задании, вам предлагается аналитически вывести формулу для вычисления расстояния Кульбака — Лейблера между нормальным распределением с параметрами $\mu$, $\sigma$ (скаляры) и стандартным нормальным распределением (с параметрами $0$, $1$).

$$
\large\mathrm{KL}\left(\mathcal{N}\left(\mu, \sigma^2\right),\,\mathcal{N}\left(0, 1\right)\right)
$$

В ходе аналитического вывода, вам могут понадобиться следующие формулы:
- Определение KL-дивергенции
  $$\large\mathrm{KL}\left(p,\,q\right) = \int_{-\infty}^{+\infty} p(x) \log\left(\frac{p(x)}{q(x)}\right) \mathrm{d}x$$
- Формула плотности нормального распределения
  $$\large\mathcal{N}\left(\mu, \sigma^2\right) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2}{\left(\frac{x-\mu}{\sigma}\right)}^2}$$
- Формула математического ожидания абсолютно непрерывной случайной величины (и её преобразований)
  $$\large\mathbb{E}\left[f(x)\right] = \int_{-\infty}^{+\infty} f(x) p(x) \mathrm{d}x$$
  где $p(x)$ - плотность распределения непрерывной случайной величины $x$, а $f(x)$ - любая борелевская функция
- Две формулы дисперсии случайной величины
  $$\large\mathbb{D}x = \mathbb{E}\left[\left(x - \mathbb{E}x\right)^2\right] \qquad \small\text{и} \large\qquad \mathbb{D}x = \mathbb{E}\left[x^2\right] - \left(\mathbb{E}x\right)^2$$

<br/>

Используя аналитически выведенную формулу, реализуйте функцию `normal_distribution_kl_divergence`. <br/>
**Не** используйте при этом классы/функции из модуля `torch.distributions`.

In [ ]:
def normal_distribution_kl_divergence(mu, sigma):
    # \/ Your code here \/
    # /\ Your code here /\
    return kl_loss

Проверьте вашу реализацию:

In [ ]:
def test_normal_distribution_kl_divergence():
    mu_value = torch.randn(123, 321)
    sigma_value = torch.exp(torch.randn(123, 321))

    # Compute reference values
    mu_ref = torch.autograd.Variable(mu_value, requires_grad=True)
    sigma_ref = torch.autograd.Variable(sigma_value, requires_grad=True)
    kl_loss_ref = torch.distributions.kl_divergence(
        torch.distributions.Normal(mu_ref, sigma_ref),
        torch.distributions.Normal(torch.zeros_like(mu_ref), torch.ones_like(sigma_ref)),
    ).mean(axis=0).sum()
    kl_loss_ref.backward()
    mu_ref_grad = mu_ref.grad
    sigma_ref_grad = sigma_ref.grad

    # Check student implementation
    mu = torch.autograd.Variable(mu_value, requires_grad=True)
    sigma = torch.autograd.Variable(sigma_value, requires_grad=True)
    kl_loss = normal_distribution_kl_divergence(mu, sigma)

    # The function must produce the same values as the library implementation
    assert torch.allclose(kl_loss, kl_loss_ref)

    # The function must be differentiable
    assert mu.grad is None
    assert sigma.grad is None
    kl_loss.backward()
    assert torch.allclose(mu.grad, mu_ref_grad)
    assert torch.allclose(sigma.grad, sigma_ref_grad)

test_normal_distribution_kl_divergence()

### Репараметризация случайных величин

Последний нюанс в реализации вариационного автоэнкодера кроется в самом процессе сэмплирования случайных величин. На данный момент, прямой проход для вариационного автоэнкодера может быть расписан следующим образом:

1. из изображения $\mathbf{x}$ извлекается вектор признаков $\varphi$ <br/>
    $\varphi = \mathrm{ENC}\left(\mathbf{x}\right)$
2. с помощью извлеченных признаков вычисляются параметры распределения <br/>
    $\mu = f_{\mu}\left(\varphi\right)$ <br/>
    $\sigma = f_{\sigma}\left(\varphi\right)$
3. латентный вектор сэмплируется из заданного распределения <br/>
    $\mathbf{z} \sim \mathcal{N}\left(\mu,\,\sigma^2\right)$
4. латентный вектор декодируется в реконструкцию изображения $\mathbf{x}'$ <br/>
    $\mathbf{x}' = \mathrm{DEC}\left(\mathbf{z}\right)$

Однако такая формулировка прямого прохода не позволяет обучать энкодер, потому что операция сэмплирования случайной величины из распределения "$\sim$" в шаге 3 - не дифференцируема. Чтобы убрать не дифференцируемую операцию "$\sim$" с пути градиента, используется прием репараметризации случайных величин.

Вместо того чтобы напрямую сэмплировать $\mathbf{z} \sim \mathcal{N}\left(\mu,\,\sigma^2\right)$, можно сэмплировать $\mathbf{z}'$ из стандартного нормального распределения $\mathbf{z}' \sim \mathcal{N}\left(0,\,1\right)$, а затем [привести](https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5#%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85_%D0%BF%D1%81%D0%B5%D0%B2%D0%B4%D0%BE%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D1%85_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD) полученное случайное значение $\mathbf{z}'$ к случайному значению $\mathbf{z}$ с интересующими нас параметрами $\mu$ и $\sigma$. После данной замены, $\mathbf{z}$ должно выражаться как дифференцируемая функция от $\mathbf{z}'$, $\sigma$ и $\mu$.

Реализуйте функцию `differentiable_sample_normal`, в которой производится описанное вычисление.
Для сэмплирования случайных величин из стандартного нормального распределения, используйте функцию `torch.randn_like`.
Обратите внимание, что на вход `differentiable_sample_normal` передается логарифм `sigma` (это сделано для численной стабильности и чтобы `sigma` не могла принимать отрицательные значения).

In [ ]:
def differentiable_sample_normal(mu, log_sigma):
    # \/ Your code here \/
    # /\ Your code here /\
    return sample, sigma

Проверьте вашу реализацию:

In [ ]:
def test_differentiable_sample_normal():
    current_seed = torch.seed()
    random_values = torch.randn(100)
    zeros_mu = torch.autograd.Variable(torch.zeros_like(random_values), requires_grad=True)
    zeros_log_sigma = torch.autograd.Variable(torch.zeros_like(random_values), requires_grad=True)

    torch.manual_seed(current_seed)
    sample, sigma = differentiable_sample_normal(zeros_mu, zeros_log_sigma)

    # The random values must be obtained from torch.randn/torch.randn_like
    assert torch.allclose(sample, random_values)
    # log_sigma=0 -> sigma=1
    assert torch.allclose(sigma, torch.ones_like(zeros_log_sigma))
    # The function must be differentiable
    assert zeros_mu.grad is None
    assert zeros_log_sigma.grad is None
    sample.sum().backward()
    assert torch.allclose(zeros_mu.grad, torch.ones_like(zeros_mu))
    assert torch.allclose(zeros_log_sigma.grad, random_values)

test_differentiable_sample_normal()

### Детали реализации

Внимательно рассмотрите код предоставленной реализации сверточного вариационного автоэнкодера.
Особое внимание обратите на функции `forward` и `step`.
Посмотрите, как здесь используются реализованные вами функции `normal_distribution_kl_divergence` и `differentiable_sample_normal`.

In [ ]:
class CelebAVariationalAutoEncoder(CelebAAutoEncoder):
    def __init__(self, kl_coeff=0.00005, **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()
        self.f_mu = torch.nn.Linear(self.hparams.latent_dim, self.hparams.latent_dim)
        self.f_log_sigma = torch.nn.Linear(self.hparams.latent_dim, self.hparams.latent_dim)
        # initialize params so that self.f_mu(X) == X
        torch.nn.init.eye_(self.f_mu.weight)
        torch.nn.init.zeros_(self.f_mu.bias)
        # initialize params so that self.f_log_sigma(X) == -1
        torch.nn.init.zeros_(self.f_log_sigma.weight)
        torch.nn.init.constant_(self.f_log_sigma.bias, -1)

    def forward(self, x):
        features = self.encoder(x)
        
        mu = self.f_mu(features)
        log_sigma = self.f_log_sigma(features)

        if self.training:
            z, sigma = differentiable_sample_normal(mu, log_sigma)
        else:
            # During validation evaluate
            # encoder/decoder without noise
            z, sigma = mu, torch.exp(log_sigma)
        
        return z, self.decoder(z), mu, sigma

    def step(self, batch, batch_idx):
        kind = "train" if self.training else "valid"
        x, _ = batch
        z, x_rec, mu, sigma = self.forward(x)
        
        # Reconstruction loss
        rec_loss = F.mse_loss(x, x_rec)
        self.log(f"{kind}_rec_loss", rec_loss, prog_bar=True)
        
        # KL loss
        kl_loss = self.hparams.kl_coeff * normal_distribution_kl_divergence(mu, sigma)
        self.log(f"{kind}_kl_loss", kl_loss, prog_bar=True)
        
        loss = rec_loss + kl_loss
        self.log(f"{kind}_loss", loss, prog_bar=True)
        return loss

Для экономии времени на семинаре вам предлагается загрузить заранее обученную версию модели. <br/>
Также для справки приведен код, который был использован для обучения этой модели.

In [ ]:
# Create a VariationalAutoEncoder
if False:
    # Init model with pretrained encoder/decoder from previous part
    vae = CelebAVariationalAutoEncoder.load_from_checkpoint(get_latest_checkpoint("AutoEncoder"), strict=False)

    trainer = pl.Trainer(
        auto_lr_find=True,
        gpus=1,
        log_gpu_memory=True,
        callbacks=[pl.callbacks.EarlyStopping("valid_loss", patience=1, verbose=True)],
        default_root_dir="VariationalAutoEncoder",
    )
    # Tune the learning rate
    trainer.tune(vae)['lr_find'].plot()
    plt.show()
    # Fine-tune the model
    trainer.fit(vae)
else:
    vae = CelebAVariationalAutoEncoder.load_from_checkpoint("VariationalAutoEncoder.ckpt")

### Визуализация результатов

Давайте рассмотрим результаты работы обученного вариационного автоэнкодера.

Сравните полученные результаты с результатами из прошлой части.

In [ ]:
show_autoencoder_results(vae)

In [ ]:
show_autoencoder_interpolation(vae)

**❓Ответьте на вопросы:** Как изменилось качество реконструкции? Осталось ли у латентного представления свойство локальности?

### Генерация новых изображений с помощью вариационного автоэнкодера

Давайте попробуем сгенерировать новые изображения тем же способом, что и в прошлой части.

In [ ]:
mean, std = get_mean_std(vae)
show_decode_random(vae, mean, std)

**❓Ответьте на вопросы:** Получилось ли с помощью вариационного автоэнкодера убрать артефакты при генерации новых изображений? Что вы можете сказать о качестве сгенерированных изображений? Как вы думаете, почему автоэнкодеры и вариационные автоэнкодеры генерируют слегка размытые изображения? Что можно сделать, чтобы получать более изображения с более резкими границами и текстурами?

In [ ]:
# Free up some memory for the next section
del vae

---

## Часть 3: Генеративно-состязательные сети

Хотя исторически идея генеративно-состязательных сетей была придумана независимо от вариационных автоэнкодеров, в данном задании мы в образовательных целях будем рассматривать генеративно-состязательные сети как дальнейшее развитие вариационных автоэнкодеров.

В конце прошлой части мы задали вопрос <i>"Почему автоэнкодеры и вариационные автоэнкодеры генерируют слегка размытые изображения?"</i>. Одно из возможных объяснений заключается в том, что при вычислении ошибки реконструкции входного изображения мы использовали попиксельное расстояние между изображениями. Таким образом, наша ошибка реконструкции никак не учитывает пространственную информацию и как следствие вынуждает декодер "размывать" детали, в пространственном расположении которых декодер не уверен.

Чтобы лучше понять, почему использование попиксельное расстояние между изображениями приводит к таким результатам, давайте рассмотрим конкретный пример. На следующей иллюстрации все 5 изображений (b-f) имеют одинаковое попиксельное расстояние до исходного изображения (а). Это противоречит нашему "интуитивному" пониманию о схожести этих изображений. Так, например, изображение (b) получено смещением на несколько пикселей и для человека почти не отличимо от оригинала, в то время как изображения (e) и (f) очевидно сильно отличаются от оригинала.

![mse.png](https://docs.google.com/uc?export=download&id=1F8M4H_4lfkGwkJp3hNvl80yx3npmpVwb)

Теперь, когда мы обратили внимание на этот недостаток ошибки реконструкции, логично было бы попробовать заменить ошибку реконструкции на какую-нибудь другую функцию ошибки. В идеале, нам бы хотелось, чтобы эта новая функция ошибки обладала "человеческой интуицией" о том, какие особенности изображения важны, а какие - нет. Конечно, такой функции в аналитическом виде скорее всего не существует, однако мы можем получить схожий результат, заменив "человеческую интуицию" на "статистическое правдоподобие".

Если бы у нас была дифференцируемая функция, которая для произвольного изображения могла бы оценить правдоподобие этого изображения или другими словами - вероятноясть найти такое изображение в эталонной выборке, то мы бы могли использовать данную функцию при обучении декодера (цель декодера - максимизировать правдоподобие сгенерированного изображения). К счастью, мы уже знаем, как можно получить такую функцию - обучить бинарный классификатор, отличающий настоящие изображения от "поддельных", сгенерированных.

Таким образом вместо $\min\ {\left\lVert\mathrm{DEC}\left(\mathbf{z}\right) - \mathbf{x}\right\rVert}^2_2$ мы будем искать $\max\ \mathbf{p}\left(\mathrm{DEC}\left(\mathbf{z}\right)\right)$, где $\mathbf{p}$ - вероятность принадлежности к эталонному датасету. Предсказывать $\mathbf{p}$ будем с помощью сети-энкодера с дополнительным линейным слоем и сигмоидальной функцией активации (как в обычной задаче бинарной классификации).

После этой замены, мы можем дополнительно слегка упростить нашу архитектуру. Заметим, что в отличие от ошибки реконструкции, максимизация правдоподобия не сопоставляет сгенерированное изображение с каким-то конкретным эталонным изображением $\mathbf{x}$. Это значит, что оптимальные параметры для распределения $\mathbf{z}$ больше не зависят от входного изображения. Учитывая их регуляризацию с помощью дивергенции Кульбака — Лейблера, мы можем зафиксировать оптимальное распределение $\mathbf{z} \sim \mathcal{N}\left(0,\,1\right)$, не зависящее от $\mathbf{x}$.

В результате мы "поменяли местами" энкодер и декодер. Теперь первая сеть в модели - декодер, который на основании случайного шума учится генерировать правдоподобные изображения, а вторая сеть в модели - энкодер, который учится отличать "поддельные" сгенерированные изображения от настоящих примеров из датасета.

В данной конфигурации, декодер принято называть **генератором**, а энкодер - **дискриминатором**:

![gan.png](https://docs.google.com/uc?export=download&id=1-CIxil5VPgNg0W7DVqTmdUUZ3XCAMxFf)

Генеративно-состязательные сети (Generative Adversarial Network, GAN) - вид нейросетевых архитектур, состоящих из двух последовательных сетей, называемых генератором и дискриминатором. В генеративно-состязательных сетях, генератор пытается создать правдоподобно выглядящие данные, а дискриминатор пытается отличить настоящие данные от "поддельных", созданных генератором.

### Состязательная природа обучения генератора и дискриминатора

Важная деталь генеративно-состязательных сетей, которая пока была упомянута только вскользь - это состязательная природа процесса обучения таких сетей. Как можно заметить, в выше описанной архитектуре сеть-генератор и сеть-дискриминатор на самом деле имеют разные, конфликтующие цели. Чем лучше работает генератор, тем сложнее будет дискриминатору отличить поддельные изображения от настоящих. Именно из-за данной особенности, эта архитектура называется состязательной (adversarial).

Более того, в прошлом разделе мы сказали что для оценки правдоподобия изображений мы "обучим бинарный классификатор" (дискриминатор), однако на практике мы не можем *заранее* обучить и зафиксировать дискриминатор. Если зафиксировать дискриминатор, то из-за состязательной природы задачи, генератор моментально переобучится под этот конкретный зафиксированный дискриминатор.

Чтобы решить данную проблему, нам нужно попеременно обучать и генератор и дискриминатор. Тогда обе эти сети будут вынуждены постоянно улучшать свои результаты в своеобразной "гонке вооружений". В итоге алгоритм обучения генеративно-состязательной сети может быть подытожен так:

1. Шаг обучения дискриминатора (веса генератора заморожены)
    - Выбирается батч эталонных изображений $\mathbf{x}$
    - Генерируется батч поддельных изображений $\mathbf{x}' = \mathrm{GEN}\left(\mathbf{z}\right)$ (для случайных $\mathbf{z} \in \mathcal{N}\left(0, 1\right)$)
    - Дискриминатор предсказывает $\mathbf{p} = \mathrm{DIS}\left(\mathbf{x}\right)$ и $\mathbf{p}' = \mathrm{DIS}\left(\mathbf{x}'\right)$
    - Происходит обратное распространение ошибки бинарной кросс-энтропии до весов дискриминатора. <br/>
      При вычислении кросс-энтропии для $\mathbf{p}$ эталонная метка - $1$ (настоящее изображение), а для $\mathbf{p}'$ эталонная метка - $0$ (подделка).

<br/>

2. Шаг обучения генератора (веса дискриминатора заморожены)
    - Генерируется батч поддельных изображений $\mathbf{x}' = \mathrm{GEN}\left(\mathbf{z}\right)$ (для случайных $\mathbf{z} \in \mathcal{N}\left(0, 1\right)$)
    - Дискриминатор предсказывает $\mathbf{p}' = \mathrm{DIS}\left(\mathbf{x}'\right)$
    - Происходит обратное распространение ошибки бинарной кросс-энтропии до весов генератора. <br/>
      При вычислении кросс-энтропии для $\mathbf{p}'$ эталонная метка - $1$ (настоящее изображение).

Обратите внимание, что эталонные метки для $\mathbf{p}'$ при оптимизации дискриминатора и генератора - разные. Это - крайне важная деталь для понимания динамики обучения генеративно-состязательных моделей. Из-за того что дискриминатор и генератор оптимизируют разные (в каком-то смысле даже противоположные) функции ошибки, процесс обучения генеративно-состязательных моделей **нельзя считать градиентным спуском**.

Многие теоремы, утверждения и интуиции, которые верны для обычных нейросетевых моделей (обучаемых градиентным спуском) не верны для генеративно-состязательных моделей. Например, при обучении градиентным спуском, мы привыкли, что по мере обучения значение функции потерь будет постепенно уменьшаться, пока не достигнет локально оптимального значения после чего модель либо "заканчивает" свое обучения, либо начинает переобучаться. При этом если функция потерь стала увеличиваться, то значит модель расходится.

Для генеративно-состязательных моделей, функция потерь может уменьшаться, увеличиваться или оставаться неизменной в процессе обучения и однозначно судить о статусе обучения по динамике значения функции потерь (или даже отдельных слагаемых в функции потерь) нельзя. Для генеративно-состязательных моделей, значение функции потерь отражает не процесс схождения модели к оптимальному решению, а **баланс** между генератором и дискриминатором. Например, если дискриминатор и генератор становятся лучше с одинаковой скоростью, то среднее значение функции потерь не будет изменяться.

Для генеративно-состязательных моделей, "оптимальным" является решение при котором распределение $\mathrm{GEN}\left(\mathbf{z}\right)$ идеально повторяет истинное распределение $\mathbf{x}$ и соответственно дискриминатор не может отличить настоящие данные от генерируемых $\mathrm{DIS}\left(\mathbf{x}\right) = \mathrm{DIS}\left(\mathbf{x}'\right) = \frac{1}{2}$.

### Детали реализации

Внимательно рассмотрите код предоставленной реализации сверточной генеративно-состязательной сети. <br/>
Особое внимание обратите на функции `generator`, `discriminator`, `forward` и `step`.

In [ ]:
class CelebAGenerativeAdversarialNetwork(CelebAAutoEncoder):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()
        self.discriminator_fc = torch.nn.Linear(self.hparams.latent_dim, 1)

    def configure_optimizers(self):
        dis_params = list(gan.encoder.parameters()) + list(gan.discriminator_fc.parameters())
        gen_params = list(gan.decoder.parameters())
        dis = torch.optim.Adam(dis_params, lr=self.hparams.lr)
        gen = torch.optim.Adam(gen_params, lr=self.hparams.lr)
        return [dis, gen], []

    def forward(self, *args, **kwargs):
        return self.generator(*args, **kwargs)

    def generator(self, z=None, num_samples=None):
        if z is None:
            z = torch.randn(num_samples, self.hparams.latent_dim, device=self.device)
        else:
            assert num_samples is None
        # sigmoid restricts the output to range [0,1]
        x_gen = torch.sigmoid(self.decoder(z))
        return z, x_gen

    def discriminator(self, x):
        features = self.encoder(x)
        logprob = self.discriminator_fc(features).squeeze(1)
        # sigmoid(logprob)=0 means "fake image"
        # sigmoid(logprob)=1 means "real image"
        return logprob

    def step(self, batch, batch_idx, optimizer_idx=None):
        kind = "train" if self.training else "valid"
        if optimizer_idx is None:
            assert not self.training
            # Manually run both generator and discriminator paths during the validation step
            total_loss = self.step(batch, batch_idx, 0) + self.step(batch, batch_idx, 1)
            self.log(f"{kind}_loss", total_loss, prog_bar=True)
            return total_loss

        # Generate fake images
        x_real, _ = batch
        _, x_fake = self.generator(num_samples=x_real.shape[0])

        loss = []
        if optimizer_idx == 0:
            # Discriminator 
            kind += "_dis"

            # Don't pass gradient through the generator
            # when optimizing the discriminator
            x_fake = x_fake.detach()

            # Prepare ground truth data
            gt_real = torch.ones(x_real.shape[0], device=self.device)
            gt_fake = torch.zeros(x_fake.shape[0], device=self.device)

            # Forward only discriminator with real data
            logprob_real = self.discriminator(x_real)

            # Loss component for real images (discriminator only)
            real_loss = F.binary_cross_entropy_with_logits(logprob_real, gt_real)
            self.log(f"{kind}_real_loss", real_loss, prog_bar=True)
            loss.append(real_loss)
        elif optimizer_idx == 1:
            # Generator
            kind += "_gen"
    
            # Prepare ground truth data
            gt_fake = torch.ones(x_fake.shape[0], device=self.device)
        else:
            raise RuntimeError()

        # Loss component for fake images (generator and discriminator)
        # Note that during the generator optimization pass gt_fake = 1
        # while during the discriminator optimization pass gt_fake = 0
        logprob_fake = self.discriminator(x_fake)
        fake_loss = F.binary_cross_entropy_with_logits(logprob_fake, gt_fake)
        self.log(f"{kind}_fake_loss", fake_loss, prog_bar=True)
        loss.append(fake_loss)
                        
        loss = sum(loss)
        self.log(f"{kind}_loss", loss, prog_bar=True)
        return loss

Для экономии времени на семинаре вам предлагается загрузить заранее обученную версию модели. <br/>
Также для справки приведен код, который был использован для обучения этой модели.

In [ ]:
# Create a GenerativeAdversarialNetwork
if False:
    gan = CelebAGenerativeAdversarialNetwork(image_size=56)
    
    # Init model with pretrained encoder/decoder from previous part
    vae = CelebAVariationalAutoEncoder.load_from_checkpoint(get_latest_checkpoint("VariationalAutoEncoder"))
    gan.encoder = vae.encoder
    gan.decoder = vae.decoder
    del vae

    trainer = pl.Trainer(
        # auto_lr_find=True,  # Doesn't work with multiple optimizers
        max_epochs=50,
        gpus=1,
        log_gpu_memory=True,
        callbacks=[pl.callbacks.ModelCheckpoint(every_n_epochs=1, save_top_k=-1, verbose=True)],
        default_root_dir="GenerativeAdversarialNetwork",
        resume_from_checkpoint=get_latest_checkpoint("GenerativeAdversarialNetwork"),
    )
    # Fine-tune the model
    trainer.fit(gan)
else:
    gan = CelebAGenerativeAdversarialNetwork.load_from_checkpoint("GenerativeAdversarialNetwork.ckpt")

### Генерация новых изображений с помощью генеративно-состязательной сети

Давайте рассмотрим результаты работы обученной генеративно-состязательной сети.

Обратите внимание, что в отличие от автоэнкодеров, генеративно-состязательные сети моделируют только отображение из латентного пространства в пространство изображений. Это значит, что при визуализации результатов работы генеративно-состязательных сетей мы не можем сравнить сгенерированное изображение с "эталоном".

Визуализируем несколько случайных генерируемых изображений и сравним их с изображениями сгенерированными при помощи вариационного автоэнкодера в прошлой части.

In [ ]:
def show_generate_random(gan, num_images=10):
    gan.train(False)

    _, imgs = gan.generator(num_samples=num_images * num_images)
    imgs = imgs.detach().numpy()
    imgs = imgs.reshape((num_images, num_images) + imgs.shape[1:])

    show_images(imgs)

In [ ]:
show_generate_random(gan)

**❓Ответьте на вопросы:** Получилось ли с помощью генеративно-состязательной сети сделать генерируемые изображения менее размытыми? Изменилась ли при этом "реалистичность" генерируемых лиц? Вы можете объяснить, почему это происходит? Что можно сделать, чтобы исправить данную проблему?

**❓Ответьте на вопросы:** Если внимательно посмотреть на генерируемые изображения, то можно заметить, что некоторые лица встречаются несколько раз (с небольшими изменениями). Вы можете объяснить, почему это происходит? Что можно сделать, чтобы исправить данную проблему?

### Бонус: интерполяция между генерируемыми изображениями

Хотя мы и не знаем "эталонные" изображения для каждого латентного вектора, мы все еще можем попробовать линейно интерполироваться между двумя случайными векторами в латентном пространстве. Посмотрите на результаты интерполяции для нескольких пар случайных латентных векторов.

In [ ]:
def show_gan_interpolation(gan, num_images=10):
    gan.train(False)

    z0 = torch.randn(num_images, gan.hparams.latent_dim, device=gan.device)
    z1 = torch.randn(num_images, gan.hparams.latent_dim, device=gan.device)
    
    outs = []
    for lerp in np.linspace(0, 1, num_images):
        z = (1-lerp) * z0 + lerp * z1
        _, out = gan.generator(z=z)
        out = out.detach()
        outs.append(out.numpy())

    show_images(np.array(outs).swapaxes(0, 1))

In [ ]:
show_gan_interpolation(gan)

**❓Ответьте на вопросы:** Как вам кажется, осталось ли у латентного представления свойство локальности? Как вы можете объяснить данный эффект?